In [2]:
"""Train TextUMC model on first 100 claims and evaluate on next 100 claims"""

from model import (
    TextUMC,
    Claim,
    Evidence,
    umc_train,
    print_evidence_clusters,
    visualize_clusters,
    ClusteringEvaluator,
)
import json
import torch
import logging
from rich.progress import (
    Progress,
    SpinnerColumn,
    TextColumn,
    BarColumn,
    TimeElapsedColumn,
    TimeRemainingColumn,
)
from rich.console import Console
from sklearn.cluster import KMeans
import os
from datetime import datetime
from typing import List, Dict

# Initialize console and logging
console = Console()


def load_claims(data: Dict, start_idx: int, count: int) -> List[Claim]:
    """Load a specified number of claims starting from given index"""
    claims = []
    for idx in range(start_idx, min(start_idx + count, len(data))):
        claim_data = data[idx]

        # Skip claims with too few evidences
        if len(claim_data["reports"]) < 5:
            continue

        claim = Claim(
            claim_id=str(idx),
            content=claim_data["claim"],
            label=claim_data["label"],
            explanation=claim_data["explain"],
        )

        # Load evidences
        for evidence in claim_data["reports"]:
            claim.evidences.append(
                Evidence(evidence_id=evidence["report_id"], content=evidence["content"])
            )
        claims.append(claim)

    return claims

output_dir = "outputs"
run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = os.path.join(output_dir, run_id)
os.makedirs(run_dir, exist_ok=True)

# Load data
data = json.load(open("../dataset/LIAR-RAW/test.json"))

# Load training and evaluation claims
train_claims = load_claims(data, 0, 160)  # First 100 claims
eval_claims = load_claims(data, 160, 40)  # Next 100 claims

logging.info(
    f"Loaded {len(train_claims)} training claims and {len(eval_claims)} evaluation claims"
)

# Initialize model and move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextUMC().to(device)

# Initialize evaluator
evaluator = ClusteringEvaluator()



[01/23/25 02:04:25] INFO     Loaded 157 training claims and 40 evaluation claims                   ]8;id=291125;file:///tmp/ipykernel_546371/2581423891.py\2581423891.py]8;;\:]8;id=953493;file:///tmp/ipykernel_546371/2581423891.py#71\71]8;;\

In [3]:
try:
    with Progress(
        SpinnerColumn(),
        TextColumn("[progress.description]{task.description}"),
        BarColumn(),
        TextColumn("[progress.percentage]{task.percentage:>3.0f}%"),
        TimeElapsedColumn(),
        TimeRemainingColumn(),
    ) as progress:

        # Training Phase
        training_progress = progress.add_task(
            f"[cyan]Training on first {len(train_claims)} claims...", total=len(train_claims)
        )

        # Train on each claim in training set
        for claim in train_claims:
            train_task = progress.add_task(
                f"[green]Training on claim {claim.claim_id}", total=100
            )

            # Train model on current claim's evidences
            _, metrics = umc_train(
                model=model,
                evidences=claim.evidences,
                num_clusters=min(len(claim.evidences), 5),
                batch_size=32,
                num_epochs=1,
                learning_rate=2e-5,
                progress=progress,
                train_task=train_task,
            )

            progress.remove_task(train_task)
            progress.update(training_progress, advance=1)

        # Save model
        model_path = os.path.join(run_dir, "textumc_model.pt")
        torch.save(model.state_dict(), model_path)
        logging.info(f"Model saved to {model_path}")

except Exception as e:
    logging.error(f"Pipeline failed: {str(e)}", exc_info=True)
    raise

Output()

[01/23/25 02:04:27] ERROR    Pipeline failed: umc_train() got an unexpected keyword argument        ]8;id=367005;file:///tmp/ipykernel_546371/413523740.py\413523740.py]8;;\:]8;id=834778;file:///tmp/ipykernel_546371/413523740.py#43\43]8;;\
                             'progress'                                                                            
                             ╭──────────────── Traceback (most recent call last) ─────────────────╮                
                             │ in <module>:23                                                     │                
                             │                                                                    │                
                             │   20 │   │   │   )                                                 │                
                             │   21 │   │   │                                                     │                
                             │   22 │   │   │   # Train model on current claim's evidences        │                
                             │ ❱ 23 │   │   │   _, metrics = umc_train(                           │                
                             │   24 │   │   │   │   model=model,                                  │                
                             │   25 │   │   │   │   evidences=claim.evidences,                    │                
                             │   26 │   │   │   │   num_clusters=min(len(claim.evidences), 5),    │                
                             ╰────────────────────────────────────────────────────────────────────╯                
                             TypeError: umc_train() got an unexpected keyword argument 'progress'                  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   │   │   )                                                                               │
│   21 │   │   │                                                                                   │
│   22 │   │   │   # Train model on current claim's evidences                                      │
│ ❱ 23 │   │   │   _, metrics = umc_train(                                                         │
│   24 │   │   │   │   model=model,                                                                │
│   25 │   │   │   │   evidences=claim.evidences,                                                  │
│   26 │   │   │   │   num_clusters=min(len(claim.evidences), 5),                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: umc_train() got an unexpected keyword argument 'progress'

In [61]:
model

TextUMC(
  (bert_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/home/lonk/codes/thesis-test-code/outputs/20250123_000729/textumc_model.pt"
model = TextUMC().to(device)
model.load_state_dict(torch.load(model_path, weights_only=True))

<All keys matched successfully>

In [8]:
torch.cuda.empty_cache()
import random

# Load model


# Evaluate on a random claim in evaluation set
claim = eval_claims[random.randint(0, len(eval_claims) - 1)]

evidence_texts = [ev.content for ev in claim.evidences]
with torch.no_grad():
    evidence_embeddings = model(evidence_texts)
    evidence_embeddings_np = evidence_embeddings.cpu().numpy()

# Save embeddings
# for ev, emb in zip(claim.evidences, evidence_embeddings_np):
#     ev.embedding = emb

# Cluster evidences
n_clusters = min(len(evidence_texts), 5)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(evidence_embeddings_np)


cluster_labels

array([2, 1, 2, 1, 2, 2, 4, 2, 4, 4, 4, 0, 4, 2, 1, 1, 4, 3, 4, 2, 2, 1,
       4, 4, 2, 4, 4, 2, 4, 2], dtype=int32)

In [9]:
# Print evidence clusters
evidence_embeddings

tensor([[ 0.0075, -0.0084,  0.0146,  ...,  0.1716, -0.1300,  0.0040],
        [ 0.1359,  0.1233,  0.1041,  ...,  0.0930, -0.0234,  0.0043],
        [-0.0790,  0.0038, -0.0292,  ...,  0.0828, -0.2219, -0.0399],
        ...,
        [-0.0592,  0.0363, -0.0257,  ...,  0.0577, -0.1522, -0.0007],
        [-0.0299,  0.0493,  0.0067,  ...,  0.0382, -0.1866, -0.1066],
        [ 0.0372,  0.0368,  0.0315,  ...,  0.0918, -0.1743, -0.0111]],
       device='cuda:0', dtype=torch.float16)

In [10]:
evidence_embeddings_np.shape

(30, 128)